In [1]:
import os
os.chdir('/home/blueconet/Downloads/Image Classification')

In [2]:
from cifar_input import cifar10_input
import ResNet_cifar10_2 as model
import tensorflow as tf

BATCH_SIZE=16
EVAL_SIZE=1000
NUM_GPUS=8

def tower_loss(batch, scope):
    images, labels = batch
    
    logits = model.inference(images, keep_prob)
    _ = model.loss(logits, labels)
    
    losses = tf.get_collection('losses', scope)
    
    total_loss = tf.add_n(losses, name='total_loss')
    return total_loss

def average_grads(list_grads):
    grads = [(tf.reduce_mean([list_grads[j][i][0] for j in range(len(list_grads))],
                             0),
             list_grads[0][i][1]) for i in range(len(list_grads[0]))]
    return grads

In [4]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
    train_data, test_data = cifar10_input()
    
    train_batch = train_data.dist_batch(BATCH_SIZE,20000)
    test_batch = test_data.batch(200)

    x = tf.placeholder(tf.float32, [None, 32, 32, 3])
    y_ = tf.placeholder(tf.int32, [x.get_shape()[0]])
    keep_prob = tf.placeholder(tf.float32)
    lr = tf.placeholder(tf.float32)
    opt = tf.train.GradientDescentOptimizer(lr)

    with tf.device('/gpu:0'):
        logit = model.inference(x, keep_prob)
        loss = model.loss(logit, y_)
        grads = opt.compute_gradients(loss)    
    
    train_op = opt.apply_gradients(grads)
    sess = tf.Session()
    
    rate = 0.1
    
    sess.run(tf.initialize_all_variables())
    tf.train.start_queue_runners(sess=sess)
    for i in range(20):
        image_batch, label_batch = sess.run(train_batch)
        _, cross_entropy = sess.run([train_op, loss], feed_dict = {x: image_batch, y_: label_batch, keep_prob: 0.5, lr:rate})
        print("step %d, cross_entropy %g"%(i, cross_entropy))

step 0, cross_entropy 2.45214
step 1, cross_entropy 4.10281
step 2, cross_entropy 2.24775
step 3, cross_entropy 2.95043
step 4, cross_entropy 2.66258
step 5, cross_entropy 2.49784
step 6, cross_entropy 2.17614
step 7, cross_entropy 2.4988
step 8, cross_entropy 2.18497
step 9, cross_entropy 2.37807
step 10, cross_entropy 2.28986
step 11, cross_entropy 2.33945
step 12, cross_entropy 2.34309
step 13, cross_entropy 2.33396
step 14, cross_entropy 2.35946
step 15, cross_entropy 2.34344
step 16, cross_entropy 2.32447
step 17, cross_entropy 2.33603
step 18, cross_entropy 2.22282
step 19, cross_entropy 2.32089


In [4]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)

        keep_prob = tf.placeholder(tf.float32)
        lr = tf.placeholder(tf.float32)

        logit = model.inference(images, keep_prob)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        opt = tf.train.GradientDescentOptimizer(lr)

        with tf.device('/gpu:0'):
            with tf.name_scope('%s_%d' % ('tower', 0)) as scope:
                loss = tower_loss(train_batch, scope)
                grads = opt.compute_gradients(loss)    
    
        train_op = opt.apply_gradients(grads)
        sess = tf.Session()
        
        rate = 0.1
        mean_loss = 0.
        least = 10.
        count = 0.
    
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)
        for i in range(20000):
            _, cross_entropy = sess.run([train_op, loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean_loss = mean_loss + cross_entropy/50
    
            if i%50 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, mean_loss))
      
                if mean_loss > least:
                    count = count+1
                    if count > 4:
                        rate = rate/2.0
                        count = 0
                    else:
                        count = 0
                        least = mean_loss
                mean_loss=0
        
            if i%250 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                print("test accuracy %g"%(eval_accuracy))

step 50, cross_entropy nan
step 100, cross_entropy nan
step 150, cross_entropy nan
step 200, cross_entropy nan
step 250, cross_entropy nan
test accuracy 0
step 300, cross_entropy nan


KeyboardInterrupt: 

In [5]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)

        keep_prob = tf.placeholder(tf.float32)
        lr = tf.placeholder(tf.float32)

        logit = model.inference(images, keep_prob)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        opt = tf.train.GradientDescentOptimizer(lr)
        
        tower_grads = []
        tower_losses = []
        
        for i in range(NUM_GPUS):
            with tf.device('/gpu:%d' % i):
                with tf.name_scope('%s_%d' % ('tower', i)) as scope:
                    loss = tower_loss(train_batch, scope)
                    grads = opt.compute_gradients(loss)
                    tower_losses.append(loss)
                    tower_grads.append(grads)
                    
        grads = average_grads(tower_grads)
        mean_loss = tf.reduce_mean(tower_losses)
  
        train_op = opt.apply_gradients(grads)
        sess = tf.Session()
        
        rate = 0.1
        least = 10.
        mean = 0.
        count = 0.
        highest = 0.
    
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)
        for i in range(20000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))
      
                if mean > least:
                    count = count+1
                    if count > 4:
                        rate = rate/2.0
                        count = 0
                    else:
                        count = 0
                        least = mean
                
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
        print("highest accuracy: %g"%(highest))

step 199, cross_entropy 2.08345
test accuracy 0.228
step 399, cross_entropy 2.04429
test accuracy 0.23
step 599, cross_entropy 2.03662
test accuracy 0.26
step 799, cross_entropy 2.15182
test accuracy 0.275
step 999, cross_entropy 1.98105
test accuracy 0.313
step 1199, cross_entropy 1.9516
test accuracy 0.321
step 1399, cross_entropy 1.75111
test accuracy 0.323
step 1599, cross_entropy 1.93492
test accuracy 0.377
step 1799, cross_entropy 2.03134
test accuracy 0.297
step 1999, cross_entropy 1.40588
test accuracy 0.328
step 2199, cross_entropy 1.8076
test accuracy 0.395
step 2399, cross_entropy 2.00668
test accuracy 0.401
step 2599, cross_entropy 1.81571
test accuracy 0.378
step 2799, cross_entropy 1.74723
test accuracy 0.434
step 2999, cross_entropy 1.84324
test accuracy 0.374
step 3199, cross_entropy 1.41262
test accuracy 0.413
step 3399, cross_entropy 1.69301
test accuracy 0.456
step 3599, cross_entropy 1.20371
test accuracy 0.48
step 3799, cross_entropy 1.53721
test accuracy 0.477
ste

In [6]:
rate = 0.002
with tf.device('/cpu:0'):
    for i in range(10000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))          
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
    print("highest accuracy: %g"%(highest))

step 199, cross_entropy 1.02115
test accuracy 0.742
step 399, cross_entropy 0.898955
test accuracy 0.777
step 599, cross_entropy 0.296028
test accuracy 0.78
step 799, cross_entropy 0.818604
test accuracy 0.763
step 999, cross_entropy 0.75452
test accuracy 0.795
step 1199, cross_entropy 0.453273
test accuracy 0.766
step 1399, cross_entropy 0.379947
test accuracy 0.784
step 1599, cross_entropy 0.425368
test accuracy 0.767
step 1799, cross_entropy 0.574464
test accuracy 0.783
step 1999, cross_entropy 0.599274
test accuracy 0.765
step 2199, cross_entropy 0.734204
test accuracy 0.782
step 2399, cross_entropy 0.321743
test accuracy 0.788
step 2599, cross_entropy 0.602894
test accuracy 0.791
step 2799, cross_entropy 0.121252
test accuracy 0.788
step 2999, cross_entropy 0.37216
test accuracy 0.77
step 3199, cross_entropy 0.671112
test accuracy 0.766
step 3399, cross_entropy 0.263459
test accuracy 0.755
step 3599, cross_entropy 0.435797
test accuracy 0.773
step 3799, cross_entropy 0.576516
test

In [8]:
rate = 0.0001
with tf.device('/cpu:0'):
    for i in range(10000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))          
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
    print("highest accuracy: %g"%(highest))

step 199, cross_entropy 0.0948039
test accuracy 0.784
step 399, cross_entropy 0.734466
test accuracy 0.755
step 599, cross_entropy 0.354488
test accuracy 0.784
step 799, cross_entropy 0.791492
test accuracy 0.814
step 999, cross_entropy 0.297589
test accuracy 0.785
step 1199, cross_entropy 0.391419
test accuracy 0.771
step 1399, cross_entropy 0.164679
test accuracy 0.801
step 1599, cross_entropy 0.289679
test accuracy 0.793
step 1799, cross_entropy 0.287635
test accuracy 0.772
step 1999, cross_entropy 0.26414
test accuracy 0.787
step 2199, cross_entropy 0.444175
test accuracy 0.767
step 2399, cross_entropy 0.236612
test accuracy 0.789
step 2599, cross_entropy 0.402675
test accuracy 0.8
step 2799, cross_entropy 0.158675
test accuracy 0.779
step 2999, cross_entropy 0.330457
test accuracy 0.81
step 3199, cross_entropy 0.975576
test accuracy 0.803
step 3399, cross_entropy 0.462285
test accuracy 0.793
step 3599, cross_entropy 0.154716
test accuracy 0.771
step 3799, cross_entropy 0.255983
te